In [95]:
import os
import csv
import json
import time
import datetime
import pandas as pd

from pathlib import Path
from datetime import date
from dateutil.relativedelta import relativedelta

In [96]:
# VARIABLES
path_to_silver = 'Datalake/silver/'
path_to_gold = 'Datalake/gold/'
gh_data_staging = 'gh-stg.csv'
top_5 = 'top-five.csv'
longest_streak = 'longest-streak.csv'
heatmap = 'heatmap.csv'

In [97]:
#pd.set_option('display.max_rows', 2500)
df = pd.read_csv(path_to_silver+gh_data_staging)

# General Transformation

In [98]:
# 1: Cast 'date' col from string to date type
df['date'] = pd.to_datetime(df['date'])

In [99]:
# 2: Derive Date and Time col 
df['date_only'] = df['date'].dt.date
df['time_only'] = df['date'].dt.time

# Q1 Transformation

In [91]:
# 1: Get total commits by each committer
df['commit_count'] = df.groupby(['email'])['email'].transform('count')

In [92]:
# 2: Sort by 'commit_count' col in descending order.
top5_df = df[['email','commit_count']].sort_values(['commit_count'], ascending=False).drop_duplicates()

# Q1 Output

In [93]:
# Save this dataframe as top-five.csv in Gold layer
top5_df.to_csv(path_to_gold+top_5, encoding='utf-8', index=False, quoting=csv.QUOTE_ALL, escapechar='"')

# Q2 Transformation

## Reference: https://stackoverflow.com/questions/52901387/find-group-of-consecutive-dates-in-pandas-dataframe

In [384]:
# 1: Sort dataframe by email and date
longest_streak_df = df.sort_values(['email','date_only'], ascending=True)

In [385]:
# 2: Keep 1 commit per day per committer
longest_streak_df = longest_streak_df[['email','date_only']].drop_duplicates()

In [386]:
# 3: Check if dates are consecutive by comparing to its previous row and calculate the difference is 1 day. Returns True if consecutive, else False

# Compare with previous, next row to check if this is made by same committer and whether it is 1 day apart
day = pd.Timedelta('1d')
compareUp = (longest_streak_df['email'].eq(longest_streak_df['email'].shift())) & (longest_streak_df['date_only'].diff().abs() == day)
compareDown = (longest_streak_df['email'].eq(longest_streak_df['email'].shift(-1))) & (longest_streak_df['date_only'].diff(-1).abs() == day)
consecutive = compareUp | compareDown

# Save the results
longest_streak_df['cmpUp'] = compareUp
longest_streak_df['cmpDown'] = compareDown
longest_streak_df['consecutive'] = consecutive

In [387]:
# 4: Identify and Group each occurrence of consecutive commits

# Filter consecutive only
longest_streak_df = longest_streak_df[longest_streak_df['consecutive'] == True]

# Reset row index
longest_streak_df = longest_streak_df.reset_index(drop=True)

# If cmpUp and cmpDown value changes, it means the streak has broke
breakpoint = (longest_streak_df['cmpDown'].ne(longest_streak_df['cmpDown'].shift()) & longest_streak_df['cmpUp'].ne(longest_streak_df['cmpUp'].shift()) ).cumsum()

# Add back to longest_streak_df
longest_streak_df['streak_grp'] = breakpoint

In [388]:
# 5: Derive streak count for each committer for each occurrence
longest_streak_df['hot_streak'] = longest_streak_df[longest_streak_df['consecutive']==True].groupby(['email','consecutive','streak_grp'])['streak_grp'].transform('count')
longest_streak_df = longest_streak_df[['email','hot_streak']].sort_values(['hot_streak'], ascending=False).drop_duplicates()

In [389]:
# 6: Remove same streaks by same committer on different occurrence
longest_streak_df['hot_streak'] = longest_streak_df.groupby(['email'])['hot_streak'].transform('max')
longest_streak_df = longest_streak_df.sort_values(['hot_streak'], ascending=False).drop_duplicates()

# Q2 Output

In [390]:
# Save this dataframe as longest-streak.csv in Gold layer
longest_streak_df.to_csv(path_to_gold+longest_streak, encoding='utf-8', index=False, quoting=csv.QUOTE_ALL, escapechar='"')

# Q3 Transformation

In [378]:
# User-defined Function
# Returns hour block based on row input
def label_block(row):
    if 0 <= int(row) < 3:
        return '00-03'
    if 3 <= int(row) < 6:
        return '03-06'
    if 6 <= int(row) < 9:
        return '06-09'
    if 9 <= int(row) < 12:
        return '09-12'
    if 12 <= int(row) < 15:
        return '12-15'
    if 15 <= int(row) < 18:
        return '15-18'
    if 18 <= int(row) < 21:
        return '18-21'
    if 21 <= int(row) < 24:
        return '21-00'

In [379]:
# Intention is to prepare a dataset for PowerBI to display the heatmap along with the above queries

heatmap_df = df.sort_values(['date_only','time_only'])
# 1: Derive 3-hour block cols and tag Y/N for each rows
heatmap_df['hour_of_day'] = df['date'].dt.hour
heatmap_df['hour_block'] = heatmap_df['hour_of_day'].apply(label_block)

# 2: Derive 'day of week' col
heatmap_df['day_of_week'] = df['date'].dt.day_name()

# Q3 Output

In [382]:
# Save this dataframe as heatmap.csv in Gold layer
heatmap_df[['sha','hour_block','day_of_week']].to_csv(path_to_gold+heatmap, encoding='utf-8', index=False, quoting=csv.QUOTE_ALL, escapechar='"')